<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/manim/HF_Training_Script_mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
!pip install datasets transformers accelerate peft bitsandbytes wandb flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 33.5 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120352136 sha256=63ab5a3883b67719671e154beb91522e2901cbe4af0c5e031306a06a85df0be5
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash_attn


In [9]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import transformers
from datasets import load_dataset
import math
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [7]:
hf_dataset = load_dataset('pravsels/manim_pile')

Generating train split:   0%|          | 0/7751 [00:00<?, ? examples/s]

In [8]:
hf_dataset['train'], hf_dataset['eval'] = hf_dataset['train'].train_test_split(.1).values()

hf_dataset

DatasetDict({
    train: Dataset({
        features: ['metadata', 'text'],
        num_rows: 6975
    })
    eval: Dataset({
        features: ['metadata', 'text'],
        num_rows: 776
    })
})

In [9]:
hf_dataset['train'][11]

{'metadata': {'dataset_name': 'mediciresearch/manimation', 'filename': 'N/A'},
 'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: Can you create an animated video tutorial that explains how to solve a surface integral problem, including the steps of parameterizing the surface, computing the surface element, converting the integral into a more solvable form, and then finding the final solution? Specifically, can the video demonstrate solving the integral of a function over a spherical surface, starting with the integral of (x^2 + y^2) over a sphere, parameterizing the sphere using spherical coordinates (theta and phi), finding the expression for the surface element in these coordinates, substituting into the integral, and performing the integration to arrive at the final result? Additionally, can each step in the tutorial be accompanied by a written explanation on the screen that guides viewers thro

In [10]:
hf_dataset['eval'][11]

{'metadata': {'dataset_name': 'pravsels/manim_ManimCommunity_issues',
  'filename': 'N/A'},
 'text': '`SVGMobject` doesn\'t accept `str` for `color` attribute : ## Description of bug / unexpected behaviour\r\nLook at this code:\r\n```py\r\nclass Test(Scene):\r\n  def construct(self):\r\n    with open(\'a.svg\', \'w\') as f:\r\n      f.write(\'\'\'<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 256 512"><path d="M160 64c0-11.8-6.5-22.6-16.9-28.2s-23-5-32.9 1.6l-96 64C-.5 111.2-4.4 131 5.4 145.8s29.7 18.7 44.4 8.9L96 123.8V416H32c-17.7 0-32 14.3-32 32s14.3 32 32 32h96 96c17.7 0 32-14.3 32-32s-14.3-32-32-32H160V64z"/></svg>\'\'\')\r\n    self.add(SVGMobject("a.svg", color="WHITE"))\r\n```\r\n[error.log](https://github.com/ManimCommunity/manim/files/10518649/error.log)\r\nThis produces the attached traceback, which is not something expected because passing colors as string works on other places like\r\n```py\r\nclass Test(Scene):\r\n  def construct(self):\r\n    self.add(Square(color=

In [11]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# max_context_size= 16384
# setting context length of out inputs
# block_size = tokenizer.model_max_length

block_size = 1024     # context len of input
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          padding_side='left',
                                          use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
def tokenize_func(examples):
  return tokenizer(examples['text'],
                   max_length=block_size,
                   truncation=True)

In [17]:
tokenized_dataset = hf_dataset.map(tokenize_func,
                                   batched=True,
                                   num_proc=8,
                                   remove_columns=['text', 'metadata'])

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

In [18]:
print(tokenized_dataset)
print(tokenized_dataset['train'][0])
print(tokenized_dataset['eval'][0])

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6975
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 776
    })
})
{'input_ids': [32013, 2826, 19508, 11778, 23011, 750, 1242, 267, 10267, 62, 1027, 62, 30255, 3574, 1191, 1494, 2, 16335, 280, 8186, 889, 14522, 12775, 188, 185, 27, 4013, 4282, 245, 3650, 285, 3180, 1007, 6413, 280, 254, 2054, 340, 18181, 13, 1585, 29, 188, 185, 2211, 403, 8591, 11528, 1433, 254, 11778, 285, 912, 2929, 9150, 14278, 280, 13113, 1242, 254, 11988, 4739, 2145, 13, 66, 17, 79, 14522, 13, 188, 185, 188, 185, 1672, 2066, 5888, 12775, 188, 185, 27, 4013, 4282, 245, 3650, 285, 3180, 1007, 6413, 280, 852, 340, 4052, 276, 6091, 13, 1585, 29, 188, 185, 546, 11528, 1433, 254, 8655, 473, 6982, 3654, 1154, 1020, 330, 8998, 285, 3858, 29977, 1020, 330, 27333, 7802, 13, 188, 185, 188, 185, 1672, 1717, 276, 26883, 254, 3605, 188, 185, 27, 4013, 10271, 543, 245, 5833, 280,

In [19]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts

  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [20]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=128,
                                   num_proc=8)
lm_dataset

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3682
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 402
    })
})

In [21]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

input ids :  [32013, 2826, 19508, 11778, 23011]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [32013, 2826, 19508, 11778, 23011]


In [6]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(
                                              base_model,
                                              device_map="auto",
                                              torch_dtype=torch.bfloat16,
                                              attn_implementation="flash_attention_2",
                                              # load_in_8bit=True
                                             )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [16]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [17]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [49]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [59]:
training_args = TrainingArguments(
  f"{base_model}_finetuned_manim_pile_context_{block_size}",
  evaluation_strategy = "epoch",
  # num_train_epochs=10,
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=8,
  weight_decay=0.01,
  report_to="wandb",
  run_name=f"{base_model.split('/')[-1]}_manim_pile_context_{block_size}",
  push_to_hub=True
)

In [60]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['eval']
)

In [61]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
model.save_pretrained('./')

In [ ]:
trainer.push_to_hub()

In [2]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
model_name = 'pravsels/deepseek-coder-6.7b-instruct_finetuned_manim_pile_context_1024'
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          padding_side='left',
                                          use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16,
                                             attn_implementation="flash_attention_2",
                                             load_in_4bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model.load_adapter(model_name)

In [5]:
gen_context_len = 512

sys_prompt = "you a helpful AI agent that generates math animation code with the library Manim. you shouldn't be too verbose and also your your generation once the program ends. "

input_text = "write code to transition a blue circle to a red square"
inputs = tokenizer(sys_prompt + input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


RuntimeError: FlashAttention only supports Ampere GPUs or newer.